In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 5.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/공간정보활용 공모전/

/content/drive/.shortcut-targets-by-id/12CdYaiaXminsUG-MrYvqDyVBDHcUxd6B/공간정보활용 공모전


In [ ]:
from dataProcssingFunction import polygon2gray
from torchvision import transforms

import torch
import numpy as np
import pandas as pd
import geopandas as gpd

print(gpd.__version__)

0.12.2


In [ ]:
file = '/content/drive/MyDrive/공간정보활용 공모전/강원도_표준지공시지가공간정보/AL_42_D152_20220929.shp'
gw = gpd.read_file(file, encoding='euckr')
file = '/content/drive/MyDrive/공간정보활용 공모전/경기도_표준지공시지가공간정보/AL_41_D152_20220929.shp'
gg = gpd.read_file(file, encoding='euckr')

춘천시
홍천군
횡성군
원주시
평창군

가평군
남양주시
광주시
이천시
여주시

In [ ]:
gw_adj = ['춘천시', '홍천군', '횡성군', '원주시', '평창군']
df = gw['A2'].apply(lambda row: row.split()[1]== '서울')
for city in gw_adj:
  temp = gw['A2'].apply(lambda row: row.split()[1]== city)
  df = pd.merge(left=df, right=temp, how='inner', left_index=True, right_index=True)

gw = gw[df.any(axis=1)]

<ipython-input-6-3e4077436cc2>:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'A2_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(left=df, right=temp, how='inner', left_index=True, right_index=True)


In [ ]:
gg_adj = ['가평군','남양주시', '광주시', '이천시', '여주시']
df = gg['A2'].apply(lambda row: row.split()[1]== '서울')
for city in gg_adj:
  temp = gg['A2'].apply(lambda row: row.split()[1]== city)
  df = pd.merge(left=df, right=temp, how='inner', left_index=True, right_index=True)

gg = gg[df.any(axis=1)]

<ipython-input-7-d58a20e6eef9>:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'A2_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(left=df, right=temp, how='inner', left_index=True, right_index=True)


In [ ]:
yp_adj = pd.concat([gg, gw])

In [ ]:
yp_adj.dropna(subset=['A19'], inplace=True)

In [ ]:
yp_adj.A19 = yp_adj.A19.astype('int64')

In [ ]:
yp_adj.drop(yp_adj[yp_adj.geometry.type == 'MultiPolygon'].index, inplace=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
convert_tensor = transforms.ToTensor()
data = []
cnt = 0
for polygon in yp_adj.geometry:
    arr = polygon2gray(polygon, 224)
    data.append(convert_tensor(arr))
    cnt += 1
    if cnt%1000==0:
      print(cnt)
X = torch.cat(data, 0)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


In [ ]:
Y = torch.from_numpy(yp_adj.A19.to_numpy()).long().to(device)

In [ ]:
#Tensor를 numpy로 변경
#X_np = X.numpy( )
#이 때, tensor의 device type이 'cuda'이면 'cpu'로 바꿔줘야 한다.
X_np = X.cpu( ).numpy( )
Y_np = Y.cpu( ).numpy( )


#npy파일로 local에 저장
#numpy 배열 'b'를 경로에 저장
X_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/adj_X_npy_file'
Y_file = '/content/drive/MyDrive/공간정보활용 공모전/양평군/adj_Y_npy_file'
np.save(X_file, X_np)
np.save(Y_file, Y_np)